In [57]:
import pandas as pd

In [58]:
import requests

In [59]:
import numpy as np

In [60]:
from sklearn.ensemble import RandomForestClassifier

In [61]:
from sklearn.model_selection import train_test_split

In [62]:
from sklearn.metrics import accuracy_score, classification_report

In [63]:
from sklearn.preprocessing import StandardScaler

In [64]:
def moving_averages_features(data, short_window=10, mid_window=50, long_window=200):
    data['short_mavg'] = data['Close'].rolling(window=short_window, min_periods=1, center=False).mean()
    data['mid_mavg'] = data['Close'].rolling(window=mid_window, min_periods=1, center=False).mean()
    data['long_mavg'] = data['Close'].rolling(window=long_window, min_periods=1, center=False).mean()
    return data.dropna()

In [65]:
def prepare_data(data):
    X = data[['short_mavg', 'mid_mavg', 'long_mavg', 'sentiment']]
    y = np.where(data['Close'].shift(-1) > data['Close'], 1, 0)
    return X, y

In [66]:
data = pd.read_csv('/Users/manaswiniswamy/Downloads/AMZN.csv', parse_dates=True, index_col='Date')

In [67]:
data.dtypes

Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [68]:
from datetime import datetime

In [69]:
results = []

month_map = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sep':'09',
             'Oct':'10','Nov':'11','Dec':'12'}

# data = {
#     "date": [],
#     "text": [],
#     "source": [],
#     "title": [],
#     "sentiment":[]
# }

# for i in range (1,2):
#     api_url = "https://stocknewsapi.com/api/v1?tickers=AMZN&data=yeartodate&items=100&page="+ str(i) +"&token=metobjbevvotlrizynavabhyisukldmxcxsdwgrf"
#     response = requests.get(api_url)
#     response_data = response.json()
#     for rd in response_data["data"]:
#         date_text = rd["date"].split(' ')
#         data["date"].append(date_text[3] + "-" + month_map[date_text[2]] + "-" + date_text[1])
#         data["text"].append(rd["text"])
#         data["source"].append(rd["source_name"])
#         data["title"].append(rd["title"])
#         data["sentiment"].append(rd["sentiment"])
#     results += response_data["data"]
# # results
# # print(data)
# import pandas as pd
# df = pd.DataFrame(data)
# print(df)

data_history = {
    "Date": [],
    "source": [],
    "sentiment":[]
}

for i in range (1,100):
    api_url = "https://stocknewsapi.com/api/v1?tickers=AMZN&data=yeartodate&items=100&page="+ str(i) +"&token=metobjbevvotlrizynavabhyisukldmxcxsdwgrf"
    response = requests.get(api_url)
    response_data = response.json()
    for rd in response_data["data"]:
        date_text = rd["date"].split(' ')
        dt = date_text[3] + month_map[date_text[2]] + date_text[1]
#         print(type(datetime.strptime(dt, '%Y-%m-%d').date()))
        data_history["Date"].append(pd.Timestamp(dt))
#         data_history["Date"].append(date_text[3] + "-" + month_map[date_text[2]] + "-" + date_text[1])
#         data["text"].append(rd["text"])
        data_history["source"].append(rd["source_name"])
#         data["title"].append(rd["title"])
        data_history["sentiment"].append(rd["sentiment"])
    results += response_data["data"]
# results
# print(data)
import pandas as pd
df = pd.DataFrame(data_history)
print(df)

           Date           source sentiment
0    2023-08-06    InvestorPlace  Positive
1    2023-08-06           Forbes   Neutral
2    2023-08-06    Seeking Alpha  Positive
3    2023-08-06  The Motley Fool  Positive
4    2023-08-06  The Motley Fool  Positive
...         ...              ...       ...
9895 2021-07-13          Barrons  Negative
9896 2021-07-13    Yahoo Finance  Positive
9897 2021-07-13  The Motley Fool  Positive
9898 2021-07-13     Market Watch  Negative
9899 2021-07-13          Reuters  Positive

[9900 rows x 3 columns]


In [70]:
data = moving_averages_features(data)

In [71]:
df['sentiment'] = df['sentiment'].map({'Positive': 1, 'Negative': -1, 'Neutral':0})
df

,Date,source,sentiment
0,2023-08-06,InvestorPlace,1
1,2023-08-06,Forbes,0
2,2023-08-06,Seeking Alpha,1
3,2023-08-06,The Motley Fool,1
4,2023-08-06,The Motley Fool,1
...,...,...,...
9895,2021-07-13,Barrons,-1
9896,2021-07-13,Yahoo Finance,1
9897,2021-07-13,The Motley Fool,1
9898,2021-07-13,Market Watch,-1


In [72]:
resultdf=data.merge(df,how="inner",on="Date")
resultdf

,Date,Open,High,Low,Close,Adj Close,Volume,short_mavg,mid_mavg,long_mavg,source,sentiment
0,2022-08-08,142.050003,144.229996,138.289993,139.410004,139.410004,52229000,139.410004,139.410004,139.410004,PYMNTS,1
1,2022-08-08,142.050003,144.229996,138.289993,139.410004,139.410004,52229000,139.410004,139.410004,139.410004,CNBC Television,1
2,2022-08-08,142.050003,144.229996,138.289993,139.410004,139.410004,52229000,139.410004,139.410004,139.410004,Zacks Investment Research,1
3,2022-08-08,142.050003,144.229996,138.289993,139.410004,139.410004,52229000,139.410004,139.410004,139.410004,The Motley Fool,1
4,2022-08-08,142.050003,144.229996,138.289993,139.410004,139.410004,52229000,139.410004,139.410004,139.410004,The Motley Fool,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4562,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,130.860002,127.856601,105.867450,Market Watch,0
4563,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,130.860002,127.856601,105.867450,WSJ,0
4564,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,130.860002,127.856601,105.867450,TechXplore,1
4565,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,130.860002,127.856601,105.867450,Proactive Investors,1


In [73]:
X, y = prepare_data(resultdf)

In [74]:
X

,short_mavg,mid_mavg,long_mavg,sentiment
0,139.410004,139.410004,139.410004,1
1,139.410004,139.410004,139.410004,1
2,139.410004,139.410004,139.410004,1
3,139.410004,139.410004,139.410004,1
4,139.410004,139.410004,139.410004,1
...,...,...,...,...
4562,130.860002,127.856601,105.867450,0
4563,130.860002,127.856601,105.867450,0
4564,130.860002,127.856601,105.867450,1
4565,130.860002,127.856601,105.867450,1


In [75]:
len(y)

4567

In [76]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [78]:
clf = RandomForestClassifier(random_state=42)

In [79]:
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [80]:
y_pred = clf.predict(X_test)

In [81]:
accuracy = accuracy_score(y_test, y_pred)

In [82]:
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.97
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98       886
           1       0.00      0.00      0.00        28

    accuracy                           0.97       914
   macro avg       0.48      0.50      0.49       914
weighted avg       0.94      0.97      0.95       914

